In [17]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import bagofwords.ipynb
from sklearn.metrics import classification_report, accuracy_score

ModuleNotFoundError: No module named 'bagofwords'

In [3]:
from nltk.corpus import stopwords
import nltk
from nltk.stem import WordNetLemmatizer

In [ ]:
def carregando_csv():
    
    # Lendo o txt
    df = pd.read_csv(r"train.txt", sep=";")

    # Renomeando colunas
    df.columns = ["mensagem", "emocao"]

    # Transformando em csv
    df.to_csv("train.csv", index=False)

    print("Arquivo convertido e salvo como csv")
    
    return df

### Pré processamento dos Dados

In [ ]:
def processando_dados(df):

    # Retirando letras maiúsculas
    df["mensagem"] = df["mensagem"].str.lower()

    # Retirando palavras 'stopwords'
    nltk.download('stopwords')
    stop_words = set(stopwords.words('english'))

    df["mensagem"] = df["mensagem"].apply(lambda x: ' '.join(word for word in x.split() if word not in stop_words))

    nltk.download('wordnet')
    nltk.download('omw-1.4')

    lemmatizer = WordNetLemmatizer()
    df["mensagem"] = df["mensagem"].apply(lambda x: ' '.join(lemmatizer.lemmatize(word) for word in x.split()))

    return df

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Samsung\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Samsung\AppData\Roaming\nltk_data...
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\Samsung\AppData\Roaming\nltk_data...


### Main

In [ ]:
def main():

    df = carregando_csv()
    df = processando_dados(df)

    while(opcao):
        
        print('Selecione uma das técnicas para prosseguir:')
        print('1 - Bag-of-Words')
        print('2 - TF-IDF')
        print('3 - n-grams')
        print('4 - word embeddings')
        print('5 - subword embeddings')
        print('6 - BERT')
        print('7 - Ver estatísticas')
        print('8 - Sair')

        opcao = input()
    
        if opcao == '1':
            y_test, y_pred = bagofwords(df)

            print("Acurácia:", accuracy_score(y_test, y_pred))
            print("\nRelatório de Classificação:\n", classification_report(y_test, y_pred))
            

        elif opcao == '2':
            print('Em construção')

        elif opcao == '3':
            print('Em construção')

        elif opcao == '4':
            print('Em construção')

        elif opcao == '5':
            print('Em construção')

        elif opcao == '6':
            print('Em construção')

        elif opcao == '7':
            print('Em construção')

        elif opcao == '8':
            break
        
        else:
            print('Opção inválida :O tente de novo!!')
    